In [ ]:
"""
Implementation of Bi-directional LSTM-CNNs-CRF model for NER.
"""

import os
import sys
import gc
import json

current_path = os.path.dirname(os.path.realpath(__file__))
root_path = os.path.dirname(os.path.dirname(os.path.realpath(__file__)))
sys.path.append(root_path)

import time
import argparse

import numpy as np
import torch
from torch.optim.adamw import AdamW
from torch.optim import SGD
from torch.nn.utils import clip_grad_norm_
from neuronlp2.io import get_logger, conll03_data, CoNLL03Writer, iterate_data
from neuronlp2.models import BiRecurrentConv, BiVarRecurrentConv, BiRecurrentConvCRF, BiVarRecurrentConvCRF
from neuronlp2.optim import ExponentialScheduler
from neuronlp2 import utils


In [ ]:
def evaluate(output_file, scorefile):
    script = os.path.join(current_path, 'eval/conll03eval.v2')
    os.system("perl %s < %s > %s" % (script, output_file, scorefile))
    with open(scorefile, 'r') as fin:
        fin.readline()
        line = fin.readline()
        fields = line.split(";")
        acc = float(fields[0].split(":")[1].strip()[:-1])
        precision = float(fields[1].split(":")[1].strip()[:-1])
        recall = float(fields[2].split(":")[1].strip()[:-1])
        f1 = float(fields[3].split(":")[1].strip())
    return acc, precision, recall, f1

In [ ]:
def get_optimizer(parameters, optim, learning_rate, lr_decay, amsgrad, weight_decay, warmup_steps):
    if optim == 'sgd':
        optimizer = SGD(parameters, lr=learning_rate, momentum=0.9, weight_decay=weight_decay, nesterov=True)
    else:
        optimizer = AdamW(parameters, lr=learning_rate, betas=(0.9, 0.999), eps=1e-8, amsgrad=amsgrad, weight_decay=weight_decay)
    init_lr = 1e-7
    scheduler = ExponentialScheduler(optimizer, lr_decay, warmup_steps, init_lr)
    return optimizer, scheduler

In [ ]:
def eval(data, network, writer, outfile, scorefile, device):
    network.eval()
    writer.start(outfile)
    for data in iterate_data(data, 256):
        words = data['WORD'].to(device)
        chars = data['CHAR'].to(device)
        labels = data['NER'].numpy()
        masks = data['MASK'].to(device)
        postags = data['POS'].numpy()
        chunks = data['CHUNK'].numpy()
        lengths = data['LENGTH'].numpy()
        preds = network.decode(words, chars, mask=masks, leading_symbolic=conll03_data.NUM_SYMBOLIC_TAGS)
        writer.write(words.cpu().numpy(), postags, chunks, preds.cpu().numpy(), labels, lengths)
    writer.close()
    acc, precision, recall, f1 = evaluate(outfile, scorefile)
    return acc, precision, recall, f1